In [122]:
import numpy as np
import os

n = 3

# Import words from solutions_nyt.txt and nonsolutions_nyt.txt. They are quoted (e.g. "soare") and separated by ", ".
with open("solutions_nyt.txt", "r") as f:
    # Remove all spaces and quotes
    words = set(f.read().replace(" ", "").replace('"', "").split(",")[:n])
with open("nonsolutions_nyt.txt", "r") as f:
    guesses = set(f.read().replace(" ", "").replace('"', "").split(',')[:n])
guesses.update(words)
guesses = list(guesses)
words = list(words)
print(f"There are {len(words)} words and {len(guesses)} guesses.")

There are 3 words and 6 guesses.


In [123]:
# Save to cpp/src/data/wordlist.hpp and cpp/src/data/guesslist.hpp
USE_CONSTEXPR = False
DATA_FOLDER = "cpp/src/data/"

# Template for a c++ file that contains the data as a constexpr
constexpr_template = """\
#include <array>

constexpr std::array {name} = {{{words}}};
"""

# # Template for a c++ file that imports the data from a csv file at the path [DATA_FOLDER]/[name].csv
# csv_template = """\
# #include <fstream>
# #include <iostream>
# #include <string>


# int main() {{
#     std::ifstream file("{DATA_FOLDER}{name}.csv");
#     std::string line;
#     while (std::getline(file, line)) {{
#         std::cout << line << std::endl;
#     }}
# }}
# """

def save_constexpr_cpp(path, name, words, words_per_line=10):
    with open(path, "w") as f:
        words_str = []
        for i in range(0, len(words), words_per_line):
            words_str.append(", ".join(f'"{word}"' for word in words[i:i+words_per_line]))
        words_str = "\n    " + ",\n    ".join(words_str)
        f.write(constexpr_template.format(name=name, words=words_str, DATA_FOLDER=DATA_FOLDER))
        
# def save_csv_cpp(path, name, csv_path):
#     with open(path, "w") as f:
#         f.write(csv_template.format(name=name, DATA_FOLDER=DATA_FOLDER))
        
save_constexpr_cpp(os.path.join(DATA_FOLDER, "wordlist.hpp"), "words", words)
save_constexpr_cpp(os.path.join(DATA_FOLDER, "guesslist.hpp"), "guesses", guesses)
# Save as regular csv files to the same folder
np.savetxt(os.path.join(DATA_FOLDER, "wordlist.csv"), words, delimiter=",", fmt="%s")
np.savetxt(os.path.join(DATA_FOLDER, "guesslist.csv"), guesses, delimiter=",", fmt="%s")

# Save some constants to generated_constants.hpp
constants_template = """\
constexpr int WORD_LENGTH = {word_length};
const int NUM_WORDS = {num_words};
const int NUM_GUESSES = {num_guesses};
"""

with open(os.path.join(DATA_FOLDER, "generated_constants.hpp"), "w") as f:
    f.write(constants_template.format(word_length=len(words[0]), num_words=len(words), num_guesses=len(guesses)))